# Functions and methods

To create a new function in Julia,

In [1]:
function hello end

hello (generic function with 0 methods)

As seen in the return of the previous cell, `hello` has no **method**. Before adding any, let us review the different kinds of arguments one can use to pass objects to Julia functions:

1. *Positional* arguments, identified by their position,
1. *Optional* arguments, that come **after** the positional arguments and take default values,
1. *Keyword* arguments, placed after the `;` (*semi-colon*) symbol in the argument tuple.

To add a method that corresponds to `hello` being called with no argument, type:

In [2]:
# 0-argument
function hello()
    println("Hello, World!")
end

hello (generic function with 1 method)

This function is called below.

In [3]:
hello()

Hello, World!


At first, it looks like running `hello()` returned no object, but it does: in Julia, the returned value is the return value of the last line (here the result of `println("Hello, World!")` unless a `return` statement is explicitly executed.

In [4]:
typeof(println("Hello, World!"))

Hello, World!


Nothing

In other words, the 0-argument method added to the function `hello` is equivalent to
```julia
function hello()
    return println("Hello, World!")
end
```

<div class="alert alert-block alert-info">
<b>Warning</b>: This behavior can be a source of bugs when coming from the C/Fortran world. Other control statements can be used, such as <tt>continue</tt> and <tt>break</tt>.
</div>

This means that `println` returns an object of type `Nothing`. In fact, `Nothing` is a special kind a type, referred to as a *singleton type*, meaning that it's a type with a single instance. This instance is associated with the symbol `nothing` somewhere in the Julia `Base` module, with the following line somewhere
```julia
const nothing = Nothing()
```

The function `hello` also has a *singleton* type that inherents from the `Function` subtype.

In [5]:
typeof(hello)

typeof(hello) (singleton type of function hello, subtype of Function)

The function `hello`  fonction `hello` est de type `typeof(hello)`, un type *singleton* qui hérite du type `Function`.

The type `Function` is an abstract type, which itself inherits from `Any`:
```
abstract type Function <: Any end
```

Being abstract, it can't be instantiated. Furthermore, in Julia, a type (concrete or abstract) can inherent from a single abstract type (or none).

<div class="alert alert-block alert-info">
    <b>Info</b>: This limitation, which greatly simplifies the dispatch mechanism, is actually not all that bad. In fact, one can use traits (<i>Holy traits</i>) to factor code that is inherit from different abstract types. These traits are also implemented by another special type, called <tt>Type</tt>.
</div>

Finally, one could have defined the `hello` function with a 0-argument method as follows (note the use of *upper cases*, reserved for type and module names in Julia).

In [6]:
struct Bonjour <: Function end

const bonjour = Bonjour()

bonjour() = println("Bonjour tout le monde !")

(::Bonjour) (generic function with 1 method)

In [7]:
bonjour()

Bonjour tout le monde !


In [8]:
typeof(bonjour)

Bonjour

To add more methods to `hello` (note the one-line syntax, equivalent to the one with the `function`... `end` but very handy for short functions):

In [9]:
hello(x) = println("Hello, $(x)!")
hello(x, y) = println("Hello, $(x) and $(y)!")

hello (generic function with 3 methods)

The interpolation operator (`$`) converts the reference object to a string object (type `String`) and substitutes it. One can generalize this behavior using a variadic function (note the concatenation of strings performed by the `*` function):

In [10]:
hello(x...) = println("Hello, " * "$(join(Base.front(x), ", ")) and $(Base.last(x))!")

hello (generic function with 4 methods)

In [11]:
hello("Pierre", "Paul", "Jacques")

Hello, Pierre, Paul and Jacques!


<div class="alert alert-block alert-info">
<b>Info</b>: We've only used positional arguments thus far, optional and keyword arguments are left for another time.
</div>

# Multiple dispatch

So far, no type annotations have been used. **That does not mean that objects have no type in Julia**, as was seen in the `typeof(hello)` and `typeof(bonjour)` examples above. For example:

In [12]:
x = 1.
typeof(x)

Float64

If one wishes to use a particular type, its constructor can be invoked explicitly. For example, to associate the symbol `y` with a half-precision version of Pi

In [13]:
typeof(π)

Irrational{:π}

In [14]:
y = Float16(π)

Float16(3.14)

Here, the irrational number Pii was converted to `Float16` through the [promotion and conversion](https://docs.julialang.org/en/v1/manual/conversion-and-promotion/) mechanism.

To improve composability however, a implicit rule in Julia is to avoid type too strictly. Indeed, if a method is initially implemented for a `Float16`, it's very likely that it will work for `Float32`, `Float64`, maybe `Int32` or even... `Complex{Float32}`, `Complex{Float64}`...

As a matter of fact, all of them are subtypes of `Number`, so one could define the cubic power function as done below.

In [15]:
cube(x::Number) = x ^ 3

cube (generic function with 1 method)

This function will work for irrational and complex numbers...

In [16]:
cube(2//3), cube(im)

(8//27, 0 - 1im)

however it won't work for strings...

In [17]:
cube("Ho ! ")

LoadError: MethodError: no method matching cube(::String)
[0mClosest candidates are:
[0m  cube([91m::Number[39m) at In[15]:1

... even though `^` is defined for `String` when the exponent is `<: Integer`. Too bad!

In [18]:
"Ho ! " ^ 3

"Ho ! Ho ! Ho ! "

One could have allowed it by relaxing the type to
```
cube(x::Union{Number,String}) = x ^ 3
```
or... Not type at all!

But what happens when multiple methods are applicable? The method to call is determined through a set of well-defined rules that are exposed in:

<div class="csl-entry">Nardelli, F. Z., Belyakova, J., Pelenitsyn, A., Chung, B., Bezanson, J., &#38; Vitek, J. (2018). Julia subtyping: a rational reconstruction. <i>Proceedings of the ACM on Programming Languages</i>, <i>2</i>(OOPSLA), 27. https://doi.org/10.1145/3276483</div>

But some cases may still be undertermined, for example:

In [19]:
foo(x::Int, y) = 2x + y
foo(x, y::Int) = x + 2y

foo (generic function with 2 methods)

So what happens if both arguments are `Int`s?

In [20]:
foo(1, 1)

LoadError: MethodError: foo(::Int64, ::Int64) is ambiguous. Candidates:
  foo(x::Int64, y) in Main at In[19]:1
  foo(x, y::Int64) in Main at In[19]:2
Possible fix, define
  foo(::Int64, ::Int64)

Fortunately, the error message is pretty descriptive, and indeed adding a method with the following signature solves the problem.
```
foo(x::Int, y::Int)
```

# Data types

There exists a handful of data types, the list of which is presented on the [dedicated page](https://docs.julialang.org/en/v1/manual/types/) from the [official documentation](https://docs.julialang.org/).